In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

*Estimation d'utilisation : moins d'une minute sur un processeur Eagle r3 (REMARQUE : Il s'agit uniquement d'une estimation. Votre temps d'exécution peut varier.)*

## Contexte

Ce tutoriel montre comment construire un `Qiskit pattern` pour évaluer les entrées d'une matrice de noyau quantique utilisée pour la classification binaire. Pour plus d'informations sur les `Qiskit patterns` et comment `Qiskit Serverless` peut être utilisé pour les déployer dans le cloud pour une exécution gérée, consultez notre [page de documentation sur IBM Quantum&reg; Platform](/guides/serverless).

## Prérequis

Avant de commencer ce tutoriel, assurez-vous d'avoir installé les éléments suivants :
- Qiskit SDK v1.0 ou ultérieur, avec le support de [visualisation](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.22 ou ultérieur (`pip install qiskit-ibm-runtime`)

## Configuration

In [2]:
!wget https://raw.githubusercontent.com/qiskit-community/prototype-quantum-kernel-training/main/data/dataset_graph7.csv

# General Imports and helper functions

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from qiskit.circuit import Parameter, ParameterVector, QuantumCircuit
from qiskit.circuit.library import UnitaryOverlap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService, Sampler

# from qiskit_serverless import IBMServerlessClient, QiskitFunction
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction


def visualize_counts(res_counts, num_qubits, num_shots):
    """Visualize the outputs from the Qiskit Sampler primitive."""
    zero_prob = res_counts.get(0, 0.0)
    top_10 = dict(
        sorted(res_counts.items(), key=lambda item: item[1], reverse=True)[
            :10
        ]
    )
    top_10.update({0: zero_prob})
    by_key = dict(sorted(top_10.items(), key=lambda item: item[0]))
    x_vals, y_vals = list(zip(*by_key.items()))
    x_vals = [bin(x_val)[2:].zfill(num_qubits) for x_val in x_vals]
    y_vals_prob = []
    for t in range(len(y_vals)):
        y_vals_prob.append(y_vals[t] / num_shots)
    y_vals = y_vals_prob
    plt.bar(x_vals, y_vals)
    plt.xticks(rotation=75)
    plt.title("Results of sampling")
    plt.xlabel("Measured bitstring")
    plt.ylabel("Probability")
    plt.show()


def get_training_data():
    """Read the training data."""
    df = pd.read_csv("dataset_graph7.csv", sep=",", header=None)
    training_data = df.values[:20, :]
    ind = np.argsort(training_data[:, -1])
    X_train = training_data[ind][:, :-1]

    return X_train



7[Files: 0  Bytes: 0  [0 B/s] Re]87[https://raw.githubusercontent.]87Saving 'dataset_graph7.csv.1'
87dataset_graph7.csv.1 100% [=============================>]   20.25K    --.-KB/s87HTTP response 200  [https://raw.githubusercontent.com/qiskit-community/prototype-quantum-kernel-training/main/data/dataset_graph7.csv]
87dataset_graph7.csv.1 100% [=============================>]   20.25K    --.-KB/s87[Files: 1  Bytes: 20.25K [93.33]8

## Step 1: Map classical inputs to a quantum problem

*   Input: Training dataset.
*   Output: Abstract circuit for calculating a kernel matrix entry.

Create the quantum circuit used to evaluate one entry in the kernel matrix. We use the input data to determine the rotation angles for the circuit's parametrized gates. We will use data samples `x1=14` and `x2=19`.

***Note: The dataset used in this tutorial can be downloaded [here](https://github.com/qiskit-community/prototype-quantum-kernel-training/blob/main/data/dataset_graph7.csv).***

In [3]:
# Prepare training data
X_train = get_training_data()

# Empty kernel matrix
num_samples = np.shape(X_train)[0]
kernel_matrix = np.full((num_samples, num_samples), np.nan)

# Prepare feature map for computing overlap
num_features = np.shape(X_train)[1]
num_qubits = int(num_features / 2)
entangler_map = [[0, 2], [3, 4], [2, 5], [1, 4], [2, 3], [4, 6]]
fm = QuantumCircuit(num_qubits)
training_param = Parameter("θ")
feature_params = ParameterVector("x", num_qubits * 2)
fm.ry(training_param, fm.qubits)
for cz in entangler_map:
    fm.cz(cz[0], cz[1])
for i in range(num_qubits):
    fm.rz(-2 * feature_params[2 * i + 1], i)
    fm.rx(-2 * feature_params[2 * i], i)

# Assign tunable parameter to known optimal value and set the data params for first two samples
x1 = 14
x2 = 19
unitary1 = fm.assign_parameters(list(X_train[x1]) + [np.pi / 2])
unitary2 = fm.assign_parameters(list(X_train[x2]) + [np.pi / 2])

# Create the overlap circuit
overlap_circ = UnitaryOverlap(unitary1, unitary2)
overlap_circ.measure_all()
overlap_circ.draw("mpl", scale=0.6, style="iqp")

<Image src="../docs/images/tutorials/quantum-kernel-training/extracted-outputs/70d6faff-9a56-44bb-b26f-f573a8c90889-0.avif" alt="Output of the previous code cell" />

## Étape 1 : Transposer les entrées classiques en un problème quantique
*   Entrée : Jeu de données d'entraînement.
*   Sortie : Circuit abstrait pour le calcul d'une entrée de la matrice de noyau.

Créez le circuit quantique utilisé pour évaluer une entrée dans la matrice de noyau. Nous utilisons les données d'entrée pour déterminer les angles de rotation des portes paramétrées du circuit. Nous utiliserons les échantillons de données `x1=14` et `x2=19`.

***Remarque : Le jeu de données utilisé dans ce tutoriel peut être téléchargé [ici](https://github.com/qiskit-community/prototype-quantum-kernel-training/blob/main/data/dataset_graph7.csv).***

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=overlap_circ.num_qubits
)
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
overlap_ibm = pm.run(overlap_circ)
overlap_ibm.draw("mpl", scale=0.6, idle_wires=False, fold=-1, style="iqp")

<Image src="../docs/images/tutorials/quantum-kernel-training/extracted-outputs/49607b34-9723-493d-85da-bd97c1351104-0.avif" alt="Output of the previous code cell" />

![Sortie de la cellule de code précédente](../docs/images/tutorials/quantum-kernel-training/extracted-outputs/70d6faff-9a56-44bb-b26f-f573a8c90889-0.avif)

## Étape 2 : Optimiser le problème pour l'exécution sur du matériel quantique
*   Entrée : Circuit abstrait, non optimisé pour un backend particulier
*   Sortie : Circuit cible et observable, optimisés pour le QPU sélectionné

Utilisez la fonction `generate_preset_pass_manager` de Qiskit pour spécifier une routine d'optimisation de notre circuit par rapport au QPU sur lequel nous prévoyons d'exécuter l'expérience. Nous définissons `optimization_level=3`, ce qui signifie que nous utiliserons le gestionnaire de passes prédéfini offrant le plus haut niveau d'optimisation.

In [6]:
num_shots = 10_000

## Evaluate the problem using statevector-based primitives from Qiskit
# from qiskit.primitives import StatevectorSampler

# sampler = StatevectorSampler()
# results = sampler.run([overlap_circ]).result()
# counts = results[0].data.meas.get_int_counts()

# Evaluate the problem using a QPU via Qiskit IBM Runtime

sampler = Sampler(mode=backend)
results = sampler.run([overlap_ibm]).result()
counts = results[0].data.meas.get_int_counts()

visualize_counts(counts, num_qubits, num_shots)

<Image src="../docs/images/tutorials/quantum-kernel-training/extracted-outputs/d2f4f6cf-067e-4d53-aa04-7ca9c803d3e1-0.avif" alt="Output of the previous code cell" />

![Sortie de la cellule de code précédente](../docs/images/tutorials/quantum-kernel-training/extracted-outputs/49607b34-9723-493d-85da-bd97c1351104-0.avif)

## Étape 3 : Exécuter à l'aide des primitives Qiskit
*   Entrée : Circuit cible
*   Sortie : Distribution de quasi-probabilités

Utilisez la primitive `Sampler` de Qiskit Runtime pour reconstruire une distribution de quasi-probabilités des états obtenus par échantillonnage du circuit. Pour la tâche de génération d'une matrice de noyau, nous nous intéressons particulièrement à la probabilité de mesurer l'état |0>.

Pour cette démonstration, nous exécuterons sur un QPU avec les primitives `qiskit-ibm-runtime`. Pour exécuter sur les primitives basées sur le vecteur d'état de `qiskit`, remplacez le bloc de code utilisant les primitives Qiskit IBM&reg; Runtime par le bloc commenté.

In [7]:
# Calculate the fidelity, or the probability to measure 0
kernel_matrix[x1, x2] = counts.get(0, 0.0) / num_shots
print(f"Fidelity: {kernel_matrix[x1, x2]}")

Fidelity: 0.1279


![Sortie de la cellule de code précédente](../docs/images/tutorials/quantum-kernel-training/extracted-outputs/d2f4f6cf-067e-4d53-aa04-7ca9c803d3e1-0.avif)

## Étape 4 : Post-traiter et renvoyer le résultat dans le format classique souhaité

*   Entrée : Distribution de probabilités
*   Sortie : Un seul élément de la matrice de noyau

Calculez la probabilité de mesurer |0> sur le circuit de recouvrement, et remplissez la matrice de noyau à la position correspondant aux échantillons représentés par ce circuit de recouvrement particulier (ligne 15, colonne 20). Dans cette visualisation, un rouge plus foncé indique des fidélités plus proches de 1.0. Pour remplir l'intégralité de la matrice de noyau, nous devons exécuter une expérience quantique pour chaque entrée.

In [ ]:
serverless = QiskitServerless()

kernel_entry_pattern = QiskitFunction(
    title="generate-kernel-entry",
    entrypoint="generate_kernel_entry.py",
    working_dir="./source/",
)

serverless.upload(kernel_entry_pattern)

## Run the Qiskit pattern as a managed service

Once we have uploaded the pattern to the cloud, we can easily run it using the `IBMServerlessProvider` client. For simplicity, we will use an exact quantum simulator in the cloud environment, so the fidelity we calculate will be exact.

In [ ]:
generate_kernel_entry = serverless.load("generate-kernel-entry")
job = generate_kernel_entry.run(
    sample1=list(X_train[x1]), sample2=list(X_train[x2])
)

kernel_matrix[x1, x2] = job.result()["fidelity"]
print(f"fidelity: {kernel_matrix[x1, x2]}")

![kernel_matrix.png](../docs/images/tutorials/quantum-kernel-training/kernel_matrix.avif)
## Déployer le patron Qiskit dans le cloud
Pour ce faire, déplacez le code source ci-dessus dans un fichier, `./source/generate_kernel_entry.py`, encapsulez le code dans un script qui prend des entrées et renvoie la solution finale, puis téléversez-le vers un cluster distant à l'aide de la classe `QiskitFunction` de `Qiskit Serverless`. Pour des conseils sur la spécification des dépendances externes, le passage d'arguments d'entrée et plus encore, consultez les [guides Qiskit Serverless](https://qiskit.github.io/qiskit-serverless/getting_started/index.html).

L'entrée du patron est une paire d'échantillons de données, `x1` et `x2`. La sortie est la fidélité entre les deux échantillons. Cette valeur sera utilisée pour remplir l'entrée de la matrice de noyau correspondant à ces deux échantillons.